### 如何用Python和深度神经网络锁定即将流失的客户？ - 简书
https://www.jianshu.com/p/7c08e9d79514

### 1. load the data

In [1]:
import numpy as np
import pandas as pd

csv_file = pd.read_csv('customer.csv')

type(csv_file)




pandas.core.frame.DataFrame

In [2]:
# .head() is a function of DataFrame
csv_file.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### 2. handcrafted feature selection

In [3]:
x_data = csv_file.loc[:,['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]

x_data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
type(x_data)


pandas.core.frame.DataFrame

In [5]:

y_data = csv_file['Exited']

# csv_file.Exited

y_data.head()



0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [6]:
type(y_data)

pandas.core.series.Series

### use labelEncoder()

In [7]:
# 在Scikit-learn工具包里面，专门提供了方便的工具LabelEncoder，让我们可以方便地将类别信息变成数值。

from sklearn.preprocessing import LabelEncoder

# 我们需要转换两列，所以建立了两个不同的labelencoder。转换的函数叫做fit_transform。

label_encoder = LabelEncoder()

x_data.Geography= label_encoder.fit_transform(x_data.Geography)

x_data.Gender = label_encoder.fit_transform(x_data.Gender)



In [8]:
x_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


### use OneHotEncoder()

In [9]:
# OneHotEncoder 可以把类别的 取值 转变为 多个变量组合 表示

# Import OneHotEncoder from sklearn.preprocessing.

# for dataframe containing multi class features, a further step of OneHotEncoder is needed. 


from sklearn.preprocessing import OneHotEncoder


# instantiate
# Create a OneHotEncoder object by using OneHotEncoder constructor
# categorical_features='all'(all feature are categorical).
# transform the 1st column(starts from 0, so here is column "Geography") of the table to an array.

one_hot_encoder = OneHotEncoder(categorical_features = [1])

### use the function to one hot the feature
array_features = one_hot_encoder.fit_transform(x_data).toarray()



In [10]:

# Note that the output is a numpy array, not a dataframe. 

type(array_features)

numpy.ndarray

In [11]:
array_features[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.1900000e+02,
       0.0000000e+00, 4.2000000e+01, 2.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0134888e+05])

In [12]:
# 我们删掉第0列, 避免掉进坑里。“虚拟变量陷阱”（dummy variable trap）。

deleted_array_feature = np.delete(array_features, [0], 1)

# 再次打印第一行：
deleted_array_feature[0]

type(deleted_array_feature)
deleted_array_feature.shape

(10000, 11)

In [13]:
y_data.head()

type(y_data)

pandas.core.series.Series

In [14]:
# 它是一个行向量，我们需要把它先转换成为列向量。你可以想象成把它“竖过来”。

y_data= y_data[:, np.newaxis]
y_data
type(y_data)

# what is ndarray

numpy.ndarray

In [15]:
# how many rows of an array?
len(y_data)
y_data.shape

(10000, 1)

In [16]:

onehotencoder = OneHotEncoder()
y_data = onehotencoder.fit_transform(y_data).toarray()

In [17]:
y_data
y_data.shape

(10000, 2)

In [18]:
print("-"*100)
print('split the data')

----------------------------------------------------------------------------------------------------
split the data


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(deleted_array_feature, y_data, test_size = 0.2, random_state = 0)


In [20]:
len(X_train)

8000

In [21]:
len(X_test)

2000

In [22]:
print("-"*100)
print('把数据进行标准化处理')

----------------------------------------------------------------------------------------------------
把数据进行标准化处理


In [23]:
X_train

array([[0.0000000e+00, 1.0000000e+00, 6.6700000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.6383064e+05],
       [1.0000000e+00, 0.0000000e+00, 4.2700000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 5.7098000e+04],
       [0.0000000e+00, 0.0000000e+00, 5.3500000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.8563076e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.3800000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.8142987e+05],
       [0.0000000e+00, 1.0000000e+00, 5.9000000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.4875016e+05],
       [1.0000000e+00, 0.0000000e+00, 6.2300000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1885526e+05]])

In [24]:

# 把数据进行标准化处理
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# 注意，我们只对特征矩阵做标准化，标签是不能动的。另外训练集和测试集需要按照统一的标准变化。

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

In [26]:
print("-"*100)
print('laoding the first module')

----------------------------------------------------------------------------------------------------
laoding the first module


In [27]:
# load the module
from sklearn import tree

clf = tree.DecisionTreeClassifier()

# train the module, get the parameters

clf = clf.fit(X_train, y_train)

In [28]:
# predict 
y_pred = clf.predict(X_test)

In [29]:
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.89      0.86      0.87      1595
          1       0.51      0.57      0.54       405

avg / total       0.81      0.80      0.80      2000



In [31]:
print("-"*100)
print('laoding the second module')

----------------------------------------------------------------------------------------------------
laoding the second module


In [32]:
import tflearn

In [33]:
net = tflearn.input_data(shape=[None, 11])

In [34]:
net = tflearn.fully_connected(net, 6, activation='relu')
net = tflearn.fully_connected(net, 6, activation='relu')
net = tflearn.fully_connected(net, 6, activation='relu')


In [35]:
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [36]:
model = tflearn.DNN(net)

In [37]:
model.fit(X_train, y_train, n_epoch=30, batch_size=32, show_metric=True)

Training Step: 7499  | total loss: 0.37028 | time: 0.540s
| Adam | epoch: 030 | loss: 0.37028 - acc: 0.8469 -- iter: 7968/8000
Training Step: 7500  | total loss: 0.38442 | time: 0.542s
| Adam | epoch: 030 | loss: 0.38442 - acc: 0.8372 -- iter: 8000/8000
--


In [38]:
X_test[0]

array([ 1.75486502, -0.57369368, -0.55204276, -1.09168714, -0.36890377,
        1.04473698,  0.8793029 , -0.92159124,  0.64259497,  0.9687384 ,
        1.61085707])

In [39]:
y_pred = model.predict(X_test)

In [40]:
y_pred[0]

array([0.79365987, 0.2063401 ], dtype=float32)

In [41]:
y_test[0]

array([1., 0.])

In [42]:
score = model.evaluate(X_test, y_test)
print('Test accuarcy: %0.4f%%' % (score[0] * 100))

Test accuarcy: 83.8000%
